
![image](../Utilities/datacleaning.png)

__Author: Christian Urcuqui__

__Date: 23 August 2018__



# Data Cleaning and Preparation


Data cleaning and preparation is the process that we would spend more time in our data science projects, and it depends of the information complexity and it's problems. In this notebook we will see the different methods in Python in order to transform our raw data in tidy data for the next analyses. 

This notebook is divided in:

